# Strade

In [18]:
import fiona
import geopandas as gpd
from fiona.transform import transform_geom

In [19]:
#----- PATH Confini ----#
PATH_CONFINI_RAW = "../Data/Raw/strade_urbane/confini-milano_raw.geojson"

#----- PATH Strade -----#
PATH_INTERSEZIONI_RAW = "../Data/Raw/strade_urbane/REGIONE_LOMBARDIA/Intersezioni_Stradali.shp"
PATH_INTERSEZIONI_STAGIN = "../Data/Staging/strade_urbane/intersezioni_stradali.geojson"
PATH_INTERSEZIONI_CLEAN = "../Data/Clean/strade_urbane/intersezioni_stradali.geojson"

## ETL

Conversione e salvataggio in Staging in formato geojson

In [20]:
src_crs = 'EPSG:32632'
dst_crs = 'EPSG:4326'

with fiona.open(PATH_INTERSEZIONI_RAW, 'r') as src:
    schema = src.schema.copy()
    crs_dst = fiona.crs.from_string(dst_crs)

    with fiona.open(PATH_INTERSEZIONI_STAGIN, 'w', driver='GeoJSON', crs=crs_dst, schema=schema) as dst:
        for feature in src:
            geom_transformed = transform_geom(src_crs, dst_crs, feature['geometry'])
            feature['geometry'] = geom_transformed
            dst.write(feature)

print(" GeoJSON convertito e salvato correttamente.")

C:\Users\aless\AppData\Local\Temp\ipykernel_8888\3880021426.py:11: FionaDeprecationWarning: instances of this class -- CRS, geometry, and feature objects -- will become immutable in fiona version 2.0
  feature['geometry'] = geom_transformed


 GeoJSON convertito e salvato correttamente.


Filtraggio sui confini di Milano

In [21]:
gdf_confini = gpd.read_file(PATH_CONFINI_RAW)
gdf_intersezioni = gpd.read_file(PATH_INTERSEZIONI_STAGIN)

dati_filtrati = gpd.sjoin(gdf_intersezioni, gdf_confini, how="inner", predicate="intersects")

Salvataggio in Clean

In [22]:
dati_filtrati.to_file(PATH_INTERSEZIONI_CLEAN, driver="GeoJSON")